<a href="https://colab.research.google.com/github/ErickMMuniz/NetworkAnalysis_TwitterTrends/blob/main/colab_burstkit_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download all datasets and codes

In [12]:
#@title **[Drive]**: Dado que estamos usando un copia de GitHub. Esto debe ser necesario para el desarrollo. No es util si no tienes las direcciones correctas que aputen a tu drive. 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
#@title **[WARNING]** : Solo usar esta celda en caso de reiniciar todo.  

#Uncomment this lines to be safe
# !rm -rf /content/data
# !rm -rf /content/burstkit
# !rm -rf /content/NetworkAnalysis_TwitterTrends/
# !rm -rf /content/README.txt
# !rm -rf /content/bin_colab.sh
# !rm -rf /content/follower_gcc.anony.dat
# !rm -rf /content/mention_gcc.anony.dat
# !rm -rf /content/retweet_gcc.anony.dat
# !rm -rf /content/timeline_tag.anony.dat
# !rm -rf /content/timeline_tag_men.anony.dat
# !rm -rf /content/timeline_tag_rt.anony.dat
# !rm -rf /content/virality2013.tar.gz

In [1]:
#@title Dataset

!wget https://zenodo.org/record/4686013/files/virality2013.tar.gz
!mkdir data
!mkdir data/virality2013.tar/
!tar -xf virality2013.tar.gz -C data/virality2013.tar/

--2022-06-14 07:48:32--  https://zenodo.org/record/4686013/files/virality2013.tar.gz
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 526346240 (502M) [application/octet-stream]
Saving to: ‘virality2013.tar.gz’

virality2013.tar.gz 100%[===================>] 501.96M  12.4MB/s    in 65s     

2022-06-14 07:49:39 (7.78 MB/s) - ‘virality2013.tar.gz’ saved [526346240/526346240]



In [2]:
#@title clone repository
!git clone https://github.com/ErickMMuniz/NetworkAnalysis_TwitterTrends.git
!mv /content/NetworkAnalysis_TwitterTrends/burstkit /content/

Cloning into 'NetworkAnalysis_TwitterTrends'...
remote: Enumerating objects: 378, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 378 (delta 73), reused 82 (delta 40), pack-reused 247
Receiving objects: 100% (378/378), 32.77 MiB | 13.79 MiB/s, done.
Resolving deltas: 100% (160/160), done.


# Project


In [ ]:
#@title Dependencias
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


In [4]:
#@title Matplotlib settings
import matplotlib.pyplot as plt

plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['font.size'] = 15
plt.rcParams['font.family'] = 'serif'
plt.rcParams['text.usetex'] = False

#mpl.rcParams['axes.prop_cycle'] = cycler(color=['r', 'g', 'b', 'y'])
#print(plt.style.available)
plt.style.use('bmh')
#plt.style.use('classic')

In [ ]:
#@title ***[WARNING]***: Siempre se debe correr esta celda para tener los mismo resultados.  
import numpy as np
from burstkit.util import SEED
np.random.seed(SEED)

# Compute principal graphs

In [8]:
#@title Some networks
import burstkit
from burstkit.calculate.graphs_attributes import calculate_some_metrics_and_return_graph_follower

#G is the network of Mutual Follow.
G = calculate_some_metrics_and_return_graph_follower()

In [4]:
#@title Some utils dataframes with timeline information. 
import burstkit
from burstkit.util.read_files import get_relation_trend_timeline_tweets

MIN_NUMBER_TWEETS = 4000

TREND_TIMELINE_TWEETS : "dict[str, pd.DataFrame]" = get_relation_trend_timeline_tweets(min_number_tweets=MIN_NUMBER_TWEETS)

100%|██████████| 1345913/1345913 [00:08<00:00, 160621.36it/s]
269it [06:17,  1.41s/it]


In [7]:
#@title La cantidad de tendencias con al menos 4000 tweets. Se reducen al considerar los RT y Menciones. 
len(TREND_TIMELINE_TWEETS.keys())

269

In [18]:
#@title Generamos un CSV con las tendencias que se calculan anterioemente. Una copia se guarda en git.
from pandas import DataFrame
PATH_INDEX_TRENDS = "/content/drive/MyDrive/tesis/data/networks_by_trend"

#TRENDS_WITH_AT_LEAST_NUMBER_TWEETS = dict(filter(lambda trend, df : df.shape[0] >= MIN_NUMBER_TWEETS ,TREND_TIMELINE_TWEETS)

index_df : "DataFrame" = DataFrame( data = {"id_trend": range(len(TREND_TIMELINE_TWEETS.keys())),"trend" : TREND_TIMELINE_TWEETS.keys() })
index_df.head()
#En este caso sí guardamos el índice.
index_df.to_csv(f"{PATH_INDEX_TRENDS}/index_trends.csv", encoding = "utf8", index = False)

In [ ]:
from pandas import DataFrame
from pandas import read_csv
from os import mkdir

from networkit.nxadapter import nx2nk
from networkit import overview
from networkit.centrality import ApproxBetweenness

PATH_INDEX_TRENDS = "/content/drive/MyDrive/tesis/data/networks_by_trend"
PATH_FOLDERS_TREND = "/content/drive/MyDrive/tesis/data/networks_by_trend"

index_trend = read_csv(f"{PATH_INDEX_TRENDS}/index_trends.csv")
index_id_trend: "dict[str, int]"  = dict(zip(index_trend["trend"], index_trend["id_trend"]))

trend = "gop"
folder_name_trend = f"TREND_ID_{index_id_trend[trend]}"
path_trend_folder = f"{PATH_FOLDERS_TREND}/{folder_name_trend}"

try:
  mkdir(path_trend_folder)
except FileExistsError:
  pass

try:
  mkdir(f"{path_trend_folder}/attributes")
except FileExistsError:
  pass



timeline_tweets_trend : 'DataFrame' = TREND_TIMELINE_TWEETS[trend]
user_trend_tweets = timeline_tweets_trend["uid"].unique()

#Get network from mutual follow
G_trend = G.subgraph(user_trend_tweets)

Git_trend = nx2nk(G_trend) 

#Guardamos el índice de los usuarios. 

index_uid_Git_trend : "DataFrame" = get_index_node_nk_to_nx(G_trend)

index_uid_Git_trend.to_csv(f"{path_trend_folder}/index_nodes.csv", index = False)

# betweenees = ApproxBetweenness(Git_trend)
# #run
# betweenees.run()

calculate_metrics_by_networkit(Git_trend)


def get_index_node_nk_to_nx(G : "nx.Graph") -> "DataFrame":
  idmap = dict((id, u) for (id, u) in zip(G.nodes(), range(G.number_of_nodes())))
  uid = idmap.keys()
  uid_part = idmap.values()
  return DataFrame( data = {"uid_part" : uid_part,
                            "uid": uid})
  
def save_attribute_by_partition(partition, name):
  values = partition.scores()
  DataFrame( data = {"uid_part" : index_uid_Git_trend["uid_part"],
                            "value": values}).to_csv(f"{path_trend_folder}/attributes/{name}.csv")


def calculate_metrics_by_networkit(G_networkit):
  
  #Betweennes
  betweenees = ApproxBetweenness(G_networkit)

  #run
  betweenees.run()

  #partiton and save to Drive
  save_attribute_by_partition(betweenees, "betweenees")


#overview(Git_trend)

1044

In [ ]:
# import networkx as nx

# nx.write_gml(G, path = "/content/drive/MyDrive/tesis/data/mutual_followers_network_with_attributes.gml")

In [ ]:
import pandas as pd
degree_attribute: "dict[str, int]" = dict(G.degree())
dict_core_number = degree_attribute
nodes = dict_core_number.keys()
value = dict_core_number.values()
df_core_number = pd.DataFrame(data={"uid":nodes,
                   "value":value})

#df_core_number.to_csv("/content/drive/MyDrive/tesis/data/mutual_follow_network/attributes/degree.csv",index=False)

In [ ]:
grerdy_communities = nx.algorithms.community.greedy_modularity_communities(G)

In [ ]:
nodes = dict_core_number.keys()
value = dict_core_number.values()
df_core_number = pd.DataFrame(data={"uid":nodes,
                   "value":value})

df_core_number.to_csv("/content/drive/MyDrive/tesis/data/mutual_follow_network/attributes/core_number.csv",index=False)

In [ ]:
from collections import Counter
degree_attribute: "dict[str, int]" = dict(G.degree())
degree_sequence = degree_attribute.values()
degree_counter = dict(Counter(degree_sequence))

X = degree_counter.keys()
Y = degree_counter.values()

import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.scatter(X,Y)
ax.set_xscale("log")
ax.set_yscale("log")
fig.show()

In [ ]:
#@title Graph k-core number vs number of Tweets

import burstkit.util.read_files as rf
TREND_TIMELINE_TWEETS : "dict[str, pd.DataFrame]" = rf.get_relation_trend_timeline_tweets()


In [ ]:
at_least_trends = filter(lambda trend_name: TREND_TIMELINE_TWEETS[trend_name].shape[0] > 4000, TREND_TIMELINE_TWEETS)
trend : "str" = list(at_.keys())[0]
timetime : "pd.DataFrame" = TREND_TIMELINE_TWEETS[trend]

In [ ]:
# F = nx.read_gml(path = "/content/drive/MyDrive/tesis/data/mutual_followers_network_with_attributes.gml")
# !cat /content/drive/MyDrive/tesis/data/mutual_followers_network_with_attributes.gml

# Danger zone

In [19]:
#@title NetworkIt install
!sudo apt install build-essential
!sudo apt install python3-dev python3-pip
!pip3 install cmake cython
!pip3 install networkit

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  python-pip-whl python3-asn1crypto python3-cffi-backend python3-crypto
  python3-cryptography python3-idna python3-keyring python3-keyrings.alt
  python3-pkg-resources python3-secretstorage python3-setuptools python3-six
  python3-wheel python3-xdg
Su

In [ ]:
from networkx import is_connected
is_connected(G)

True

In [ ]:
#Checknumber of total nodes 
import burstkit
from burstkit.util.read_files import get_mutual_followers

df_mutual_followers = get_mutual_followers()

In [ ]:
#df_mutual_followers['source'].unique()
#df_mutual_followers['target'].unique()

from pandas import concat

df_union = concat([df_mutual_followers['source'], df_mutual_followers['source'] ], axis=0)

In [ ]:
foo_unique = df_union.unique()

In [ ]:
foo_unique_int = list(map(lambda x: int(x), list(foo_unique)))
foo_unique_int.sort(reverse=False)

In [ ]:
#from itertools import Counter
#df_union.count()

from pandas import DataFrame
df_union_really_df = DataFrame( data = {"uid": df_union})

In [ ]:
!pip3 install networkx
!pip3 install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from networkit import *
Git = nxadapter.nx2nk(G)
#g = generators.HyperbolicGenerator(1e5).generate()
overview(Git)

Exception in thread _colab_inspector_thread:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_debugpy.py", line 64, in inspector_thread
    _variable_inspector.run(shell, time)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_variable_inspector.py", line 27, in run
    globals().clear()
TypeError: 'module' object is not callable



Network Properties:
nodes, edges			595460, 14273311
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.000081
clustering coefficient		0.198532
min/max/avg degree		1, 2152, 47.940453
degree assortativity		0.293644
number of connected components	1
size of largest component	595460 (100.00 %)


In [ ]:
overview(Git)

Network Properties:
nodes, edges			595460, 14273311
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.000081
clustering coefficient		0.198532
min/max/avg degree		1, 2152, 47.940453
degree assortativity		0.293644
number of connected components	1
size of largest component	595460 (100.00 %)


In [ ]:
#@title Get comunnities
%%time
from networkit.community import detectCommunities

networkit_partition = detectCommunities(Git)

/usr/local/lib/python3.7/dist-packages/networkit/stopwatch.py:49: UserWarning: networkit.Timer is deprecated, will be removed in future updates.
  warn("networkit.Timer is deprecated, will be removed in future updates.")


Communities detected in 31.10533 [s]
solution properties:
-------------------  ---------------
# communities          114
min community size       3
max community size   88576
avg. community size   5223.33
imbalance               16.9556
edge cut                 2.47395e+06
edge cut (portion)       0.173327
modularity               0.753149
-------------------  ---------------
CPU times: user 34.8 s, sys: 42.7 ms, total: 34.9 s
Wall time: 31.7 s


In [ ]:
networkit_partition.getVector()

In [ ]:
idmap = dict((id, u) for (id, u) in zip(G.nodes(), range(G.number_of_nodes())))

In [ ]:
community_module = dict( )

In [ ]:
networkit_partition = dict( zip(G.nodes(),networkit_partition.getVector() ))

In [ ]:
networkit_partition

In [ ]:
import pandas as pd
degree_attribute: "dict[str, int]" = networkit_partition
dict_core_number = degree_attribute
nodes = dict_core_number.keys()
value = dict_core_number.values()
df_core_number = pd.DataFrame(data={"uid":nodes,
                   "value":value})

df_core_number

#df_core_number.to_csv("/content/drive/MyDrive/tesis/data/mutual_follow_network/attributes/degree.csv",index=False)

,uid,value
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
595455,595455,24
595456,595456,29
595457,595457,30
595458,595458,47


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_union_really_df.set_index(['uid']).count(level='uid')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.count(level=1) should use df.groupby(level=1).count().
  """Entry point for launching an IPython kernel.


TypeError: ignored